# Cálculo de armónicos esféricos utilizando computación cuántica.   
## TFE: Computación cuántica aplciada al análisis espacial en electroencefalografía   
## Master universitario en computación cuántica (UNIR)   
### Francisco Vidal Requejo   
### Antonio José Ruz Hervás   

En este notebook se desarrolla el modelo teórico propuesto en el trabajo para la evaluación de armónicos esféricos mediante la utilización de computación cuántica.

Primeramente, es necesario instalar la librería "pennylane", que será utilizada para los circuitos cuánticos.

In [9]:
%pip install pennylane > o

Note: you may need to restart the kernel to use updated packages.


Importación de librerías utilizadas en este archivo.

In [10]:
import pennylane as qml
from pennylane import numpy as np
import math
import sympy as sp
from scipy.special import lpmv
from scipy.special import sph_harm


Ángulos utilizados para el cálculo de los armónicos esféricos. La primera coordenada corresponde con la coordenada azimutal y la segunda con la polar. Estos ángulos se corresponden con la posición de 64 electrodos de un EEG utilizando el "sistema 10-10" para elegir la posición de los mismos, considerando la cabeza como una esfera.

In [39]:
angles = [['Fc5.',-71,-21],
 ['Fc3.',-50,-28],
 ['Fc1.',-32,-45],
 ['Fcz.',23,90],
 ['Fc2.',32,45],
 ['Fc4.',50,28],
 ['Fc6.',71,21],
 ['C5..',-69,0],
 ['C3..',-46,0],
 ['C1..',23,0],
 ['Cz..',0,0],
 ['C2..',23,0],
 ['C4..',46,0],
 ['C6..',69,0],
 ['Cp5.',-71,21],
 ['Cp3.',-50,28],
 ['Cp1.',-32,45],
 ['Cpz.',23,-90],
 ['Cp2.',32,-45],
 ['Cp4.',50,-28],
 ['Cp6.',71,-21],
 ['Fp1.',-92,-72],
 ['Fpz.',92,90],
 ['Fp2.',92,72],
 ['Af7.',-92,-52],
 ['Af3.',-74,-67],
 ['Afz.',69,90],
 ['Af4.',74,67],
 ['Af8.',92,52],
 ['F7..',-92,-36],
 ['F5..',-75,-41],
 ['F3..',-60,-51],
 ['F1..',-50,-68],
 ['Fz..',46,90],
 ['F2..',50,68],
 ['F4..',60,51],
 ['F6..',75,41],
 ['F8..',92,36],
 ['Ft7.',-92,-18],
 ['Ft8.',92,18],
 ['T7..',-92,0],
 ['T8..',92,0],
 ['T9..',-115,0],
 ['T10.',115,0],
 ['Tp7.',-92,18],
 ['Tp8.',92,-18],
 ['P7..',-92,36],
 ['P5..',-75,41],
 ['P3..',-60,51],
 ['P1..',-50,68],
 ['Pz..',46,-90],
 ['P2..',50,-68],
 ['P4..',60,-51],
 ['P6..',75,-41],
 ['P8..',92,-36],
 ['Po7.',-92,54],
 ['Po3.',-74,67],
 ['Poz.',69,-90],
 ['Po4.',74,-67],
 ['Po8.',92,-54],
 ['O1..',-92,72],
 ['Oz..',92,-90],
 ['O2..',92,-72],
 ['Iz..',115,-90]
 ]

#Versión de los ángulos en radianes.
anglesRad = []
piFactor = math.pi/180
for x in angles:
    anglesRad.append([x[0], x[1] * piFactor, x[2] * piFactor])



64

In [34]:
dev = qml.device ("default.qubit", wires = nCubits)

@qml.qnode(dev)
def CircuitoCoseno(theta, nCubits = 3):
  """
  Implementar un circuito cuántico con una puerta de ratación sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar en una rotación sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  for i in range(nCubits):
    qml.RY(theta, wires = i)
  return qml.probs(wires=range(nCubits))

def CircuitoSeno(theta, nCubits=3):
  """
  Implementar un circuito cuántico con una puerta de rotación de pi/2 - theta sobre el eje Y para todos los cúbits. El mismo ángulo se aplica a todos los cúbits.
  Parámetros:
    theta: Ángulo en radianes que se quiere aplicar como una rotación pi/2 - theta sobre el eje Y.
    nCubits: Cantidad de cúbits del circuito

  Retorno:
    Array de probabilidades de cada uno de los estados.
  """
  return CircuitoCoseno((math.pi/2) - theta, nCubits)

def ExpPhi(phi, m):
  """
  Obtener el valor en coordenadas cartesianas de un número complejo en forma polar de la manera e^{im} a través de dos circuitos cuánticos de un único cúbit utilizando la igualdad e^{im} = cos(im) + i·sen(im) .

  Parámetros:
    phi: el ángulo base
    m: valor a multiplicar al ángulo.

  Retorno:
    Una tupla con el valor real e imagino del número complejo en coordenadas cartesianas.
  """
  C = CircuitoCoseno(phi * m, 1)
  S = CircuitoSeno(phi * m, 1)
  return (C[0] - C[1], S[0] - S[1])


def EvalAsocLegendre4(theta):
  """
  Evaluar los Polinomios asociados de legendre en un ángulo dado utilizando un circutio de 4 cúbits. Esto generará todos los polinomios asociados de Legendre hasta l = 4.

  Parámetros: 
    theta: ángulo en radiantes sobre el cual calcular los polinomios asociados de Legendre.

  Retorno:
    Un diccionario en el que cada elemento tiene una etiqueta de la forma "P_l_m" indicando el polinomio asociado de Legendre al que hace referencia y cuyo valor es el resultado de la evaluación de dicho polinómio sobre el ángulo dado.
  """
  nCubits = 4
  C = CircuitoCoseno(theta, nCubits)
  S = CircuitoSeno(theta, nCubits)
  PolinomiosLegendre = {'P_0_0': 1}
  PolinomiosLegendre['P_1_0']= C[0] + 2*C[1] -2*C[7] -C[15]
  PolinomiosLegendre['P_1_1']= -S[0] - 2*S[1] + 2*S[7] + S[15]
  PolinomiosLegendre['P_2_0']= (3*(C[0] - 2*C[3] + C[15]) - 1)/2
  PolinomiosLegendre['P_2_1']= 3*(C[0] + 2*C[1] - 2*C[7] - C[15])*(-S[0] -2*S[1] + 2*S[7] + S[15])
  PolinomiosLegendre['P_2_2']= 3*(S[0] - 2*S[3] + S[15])
  PolinomiosLegendre['P_3_0']= C[0] - 8*C[1] + 8*C[7] - C[15]
  PolinomiosLegendre['P_3_1']= -(3/2)*(-S[0] + 18*S[1] -18*S[7] + S[15])
  PolinomiosLegendre['P_3_2']= 15*(4*C[1] - 4*C[7])
  PolinomiosLegendre['P_3_3']= -15*(S[0] -2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_0']= (1/8)*(5*C[0] - 140*C[1] + 270*C[3] -140*C[7] + 5*C[15] + 3)
  PolinomiosLegendre['P_4_1']= -(5/2)*(7*(C[0] +2*C[1] -2*C[7] - C[15])*(4*S[1]-4*S[7]) + 3*(C[0]+2*C[1]-2*C[7]-C[15])*(-S[0]-2*S[1]+2*S[7]+S[15]))
  PolinomiosLegendre['P_4_2'] = (15/2)*(56*C[1] - 112*C[3] + 56*C[7] - S[0] - 28*S[1] + 58*S[3] - 28*S[7] - S[15])
  PolinomiosLegendre['P_4_3'] = -105*(C[0] + 2*C[1] - 2*C[7] - C[15])*(S[0] - 2*S[1] + 2*S[7] - S[15])
  PolinomiosLegendre['P_4_4'] = 105*(S[0] - 4*S[1] + 6*S[3] - 4*S[7] + S[15])

  #Polinomios de "m" negativo
  for l in range(1, nCubits + 1):
    for m in range(1, l+1):
      PolinomiosLegendre['P_' + str(l) + '_-' + str(m)] = ((-1)**(m) * (math.factorial(l-m)/math.factorial(l+m))) * PolinomiosLegendre['P_' + str(l) + '_' + str(m)]
  return PolinomiosLegendre

def EvalEsfericos4(theta, phi):
  """
  Realiza la evaluación de armónicos esféricos mediante cirucitos cuánticos de 4 cubits. Se obtiene el resultado de evalluar los armónicos esféricos para los ángulos dados hasta l = 4.

  Parámetros:
    theta: ángulo én radianes de la coordenada polar.
    phi: ángulo en radianes de la coordenada azimutal.

  Retorno:
    Un diccionario en el que cada elemento tiene una etiqueta de la forma "Y_l_m" indicando el armónico esférico al que hace referencia y cuyo valor es el resultado de la evaluación de dicho armónico esféricos sobre los ángulos dados.
  """
  l = 4
  legendre = EvalAsocLegendre4(theta)
  AE = {}
  for k in range(l+1):
    for m in range(-k, k+1):
      coef =  math.sqrt(((2*k + 1)*math.factorial(k - (m))) / (4*math.pi * math.factorial(k+(m)))) #Coeficiente de normalización
      e = ExpPhi(phi, (m))
      legendre_normalizado = (coef *  legendre['P_' + str(k) + '_' + str(m)])
      AE['Y_' + str(k) + '_' + str(m)] = ( legendre_normalizado * e[0], legendre_normalizado * e[1])
  
  return AE

In [35]:
AE = EvalEsfericos4(theta, phi)
for armonicos in AE.values():
    print(armonicos[0], armonicos[1])

0.28209479177387814 0.0
0.18296548340389993 -0.0881115329251683
0.3952877356237497 0.0
-0.18296548340389993 -0.08811153292516832
0.08320749142747604 -0.10433889404646438
0.3309876688616823 -0.15939526046205713
0.3038878129445756 0.0
-0.3309876688616823 -0.15939526046205718
0.08320749142747604 0.10433889404646438
0.018853675536764813 -0.0826033497217849
0.17810212184279134 -0.22333299684443061
0.38894232429676284 -0.18730475156132773
0.08228221038686168 0.0
-0.38894232429676284 -0.18730475156132778
0.17810212184279137 0.22333299684443067
-0.018853675536764813 -0.0826033497217849
-0.011754143143792861 -0.05149826594253829
0.045758831747021826 -0.20048254115166317
0.2580864647152372 -0.3236301904404923
0.3205626929630729 -0.15437485666760664
-0.1736900156399915 -0.0
-0.3205626929630728 -0.15437485666760667
0.25808646471523716 0.3236301904404923
-0.045758831747021826 -0.2004825411516632
-0.011754143143792865 0.0514982659425383


In [17]:
m = [0,-1,0,1,-2, -1,0,1,2,-3,-2,-1,0,1,2,3,-4,-3,-2,-1,0,1,2,3,4]
v = [0, 1,1,1, 2, 2,2,2,2, 3, 3, 3,3,3,3,3, 4, 4, 4, 4,4,4,4,4,4]
x = []
for i in range(15):
    x.append(math.cos(theta))

sph = sph_harm(m, v, phi, theta)
sph

array([ 0.28209479+0.j        ,  0.18296548-0.08811153j,
        0.39528774+0.j        , -0.18296548-0.08811153j,
        0.08320749-0.10433889j,  0.33098767-0.15939526j,
        0.30388781+0.j        , -0.33098767-0.15939526j,
        0.08320749+0.10433889j,  0.01885368-0.08260335j,
        0.17810212-0.223333j  ,  0.38894232-0.18730475j,
        0.08228221+0.j        , -0.38894232-0.18730475j,
        0.17810212+0.223333j  , -0.01885368-0.08260335j,
       -0.01175414-0.05149827j,  0.04575883-0.20048254j,
        0.25808646-0.32363019j,  0.32056269-0.15437486j,
       -0.17369002+0.j        , -0.32056269-0.15437486j,
        0.25808646+0.32363019j, -0.04575883-0.20048254j,
       -0.01175414+0.05149827j])